In [27]:
import pymc3 as pm
import pandas as pd
from ray import tune

#Se define función de entrenamiento que utilizó modelo bart_model para ajustar hiperparámetros en RayTune
def train_bart(config):
    num_trees = config["num_trees"]
    tree_depth = config["tree_depth"]
    alpha = config["alpha"]

    bart_model = bart_model(x, y, num_trees=num_trees, tree_depth=tree_depth, alpha=alpha)

    with bart_model:
        try:
            trace = pm.sample(2000, tune=1000)
            log_likelihood = -trace.log_likelihood.mean()
        except Exception as e:
            print(f"Error during sampling: {e}")
            log_likelihood = float("-inf")  # Valor predeterminado en caso de error

#métrica a optimizar:en este caso, se utiliza el log-likelihood negativo como métrica para maximizar

    return log_likelihood




In [28]:
#Definir espacio de búsqueda de hiperparámetros, para optimizar utilizando Ray Tune

config = {
    "num_trees": tune.choice([25, 50, 75]),
    "tree_depth": tune.choice([2, 3, 4]),
    "alpha": tune.choice([0.8, 0.9, 0.95]),
}


In [29]:
#Ejecución de Ray Tune para optimizar hiperparámetros (utilizando función entrenamiento y espacio de búsqueda)
import ray

if __name__ == "__main__":
    data = pd.read_csv('Datos_Bart_C.csv', delimiter=';')
    x = data[['total_credits_1', 'total_credits_2', 'total_courses_1', 'course_approved_1', 't_gpa_1']].values
    y = data['Permanencia_term_3'].values

    ray.shutdown()
    # Inicia Ray 
    ray.init()

    # Ejecución Ray Tune
    analysis = tune.run(
        train_bart,
        config=config,
        metric="log_likelihood",
        mode="max",
        num_samples=10,
        resources_per_trial={"cpu": 1},
        )

    # Obtener los mejores hiperparámetros
    best_config = analysis.get_best_config(metric="log_likelihood", mode="max")
    print("Mejores hiperparámetros:", best_config)

2023-07-28 23:46:00,561	INFO worker.py:1621 -- Started a local Ray instance.
2023-07-28 23:46:02,326	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(pid=999699) WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
2023-07-28 23:46:06,085	ERROR tune_controller.py:911 -- Trial task failed for trial train_bart_70235_00000
Traceback (most recent call last):
  File "/home/info335/.local/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2493, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(UnboundLocalError): ray::ImplicitFunc.train() (pid=999699, ip=146.83.216.133, actor_id=03d9a9d256e8f902d19f3c1b01000000, repr=tr

Trial name
train_bart_70235_00000
train_bart_70235_00001
train_bart_70235_00002
train_bart_70235_00003


2023-07-28 23:46:06,091	ERROR tune_controller.py:911 -- Trial task failed for trial train_bart_70235_00001
Traceback (most recent call last):
  File "/home/info335/.local/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2493, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(UnboundLocalError): ray::ImplicitFunc.train() (pid=999698, ip=146.83.216.133, actor_id=04ec5f2640d576477adb76ad01000000, repr=train_bart)
  File "/home/info335/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", 

TuneError: ('Trials did not complete', [train_bart_70235_00000, train_bart_70235_00001, train_bart_70235_00002, train_bart_70235_00003])

In [ ]:
#Forma de poder evaluar mejor modelo entrenado
import os

logdir = results.get_best_result("mean_accuracy", mode="max").log_dir
state_dict = torch.load(os.path.join(logdir, "model.pth"))

model = ConvNet()
model.load_state_dict(state_dict)